In [1]:
import pandas as pd
import ast
import numpy as np

In [2]:
main_data=pd.read_csv("real_domains.csv",names=["Top level domain","MX-Record from the name server","A-Record of the specific domain","Timestamp of scraping process"])

In [3]:
main_data.head()

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process
0,0--1.de,"[""mail.0--1.de"",""mxf993.netcup.net""]","[""46.38.249.145""]",2020-12-13 15:36:05
1,0--2.de,"[""mxf993.netcup.net"",""mail.0--2.de""]","[""212.227.212.163""]",2020-12-13 15:36:05
2,0-0-0-0-0.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","[""80.150.6.143""]",2020-12-13 15:36:05
3,0-0-0-1.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","[""80.150.6.143""]",2020-12-13 15:36:05
4,0-0.de,[],"[""185.53.178.13""]",2020-12-13 15:36:05


In [4]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4860885 entries, 0 to 4860884
Data columns (total 4 columns):
 #   Column                           Dtype 
---  ------                           ----- 
 0   Top level domain                 object
 1   MX-Record from the name server   object
 2   A-Record of the specific domain  object
 3   Timestamp of scraping process    object
dtypes: object(4)
memory usage: 148.3+ MB


* Top level domain: Namensserver; das, was Personen sich merken können
* Ein MX Resource Record weist einem Namen einen Mailserver zu. Er stellt eine Besonderheit dar, da er sich auf einen speziellen Dienst im Internet, nämlich die E-Mailzustellung mittels SMTP, bezieht. Alle anderen Dienste nutzen CNAME, A und AAAA Resource Records für die Namensauflösung.
* Ein A Resource Record weist einem Namen eine IPv4-Adresse zu.
* alles Object Werte

In [5]:
main_data["Top level domain"].value_counts()

powerlister.de                    1
2440.de                           1
fuehrerscheinmacher.de            1
edmm-north.de                     1
pizzeria-la-strada-auenheim.de    1
                                 ..
daimiswelt.de                     1
runtermitdenkosten.de             1
bollerwaegenerfahrung-24.de       1
techcareers.de                    1
gw-kfz-reparaturen.de             1
Name: Top level domain, Length: 4860885, dtype: int64

In [6]:
tmp_vc=main_data["Top level domain"].value_counts()
tmp_vc[tmp_vc>2]

Series([], Name: Top level domain, dtype: int64)

* es gibt keine Domain, die doppelt vorkommt

In [7]:
main_data["MX-Record from the name server"].value_counts()

[]                                                 633040
["smtpin.rzone.de"]                                509127
["localhost"]                                      303839
["mx01.kundenserver.de","mx00.kundenserver.de"]    176491
["mx00.kundenserver.de","mx01.kundenserver.de"]    176182
                                                    ...  
["mail.benjaminstrake.de"]                              1
["mail.bad-berleburg-erndtebruecker-tafel.de"]          1
["mail.xn--kdorf-jua.de"]                               1
["mail.luchterhandt.de","relay.rzone.de"]               1
["vdmaverlag.de","vdmaverlag.de"]                       1
Name: MX-Record from the name server, Length: 1294740, dtype: int64

* es gibt eine Kummulation bei bestimmten MX-Records. Das bedeutet, dass verschiedenste Domains den gleichen Mailserver nutzen (oder gar keinen).
* Teileweise gibt es mehrere MX-Records pro Domain

In [8]:
main_data[main_data["MX-Record from the name server"].isna()]

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process


* es gibt keine NaN-Werte in der Spalte MX-Record

In [9]:
main_data[~(main_data["MX-Record from the name server"].str.contains(']',case=False))]

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process
945624,derproviderserver.de,"[""dps123.derproviderserver.de"",""dps13.derprovi...","[""83.243.40.214""]",2020-12-09 20:04:42
1048480,domservice.de,"[""mail01-f.domservice.de"",""mail01-c.domservice...","[""213.179.64.5""]",2020-12-09 22:26:48
3520446,redleadsmarketing.de,"[""99smtp-v0.redleadsmarketing.de"",""99smtp-v1.r...",[],2020-12-12 08:05:05


* es gibt Zeilen in der Spalte MX-Record, die nicht korrekt übertragen wurden
* keiner dieser Spalten hat eine gechlossene eckige Klammer und kann darüber hinausgefiltert werden

In [10]:
main_data[~(main_data["MX-Record from the name server"].str.contains(']',case=False))].index

Int64Index([945624, 1048480, 3520446], dtype='int64')

In [11]:
main_data = main_data.drop([main_data.index[945624], main_data.index[1048480], main_data.index[3520446]])

In [12]:
main_data["MX-Record from the name server"]=main_data["MX-Record from the name server"].apply(lambda x: ast.literal_eval(x))

* in dem Datensatz gibt es viele überzählige Quotes, welche durch die Methode ast.literal_eval() entfernt werden

In [13]:
main_data["length_mx"] = main_data["MX-Record from the name server"].apply(lambda x: len(x))

* Die Spalte MX-Record besteht aus Listen pro Zeile
* diese Methode berechnet wie viele Werte es pro Liste gibt

In [3]:
main_data["A-Record of the specific domain"].value_counts()

["91.195.241.137"]                                    303204
[]                                                    161287
["80.150.6.143"]                                      150156
["23.236.62.147"]                                      59035
["52.58.78.16"]                                        49200
                                                       ...  
["104.27.138.20","104.27.139.20","172.67.219.104"]         1
["87.230.110.215","80.237.197.9"]                          1
["54.154.1.153"]                                           1
["45.87.80.2"]                                             1
["52.28.209.152","54.93.107.145"]                          1
Name: A-Record of the specific domain, Length: 450010, dtype: int64

* es gibt eine Kummulation bei bestimmten A-Records. Das bedeutet, dass verschiedenste Domains die gleiche IP Adresse haben(oder gar keine???).
* Teileweise gibt es mehrere A-Records pro Domain

In [15]:
main_data[main_data["A-Record of the specific domain"].isna()]

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process,length_mx
127,00359-oem-8703012-34394.de,[],NaN,2020-12-13 15:36:39,0
1090,05diner.de,[],NaN,2020-12-13 15:38:52,0
1693,089cityfashion.de,[],NaN,2020-12-13 15:40:07,0
3808,100grammseo.de,[],NaN,2020-12-13 15:43:24,0
3953,100pro-medical-food.de,[],NaN,2020-12-13 15:43:35,0
...,...,...,...,...,...
4858145,zweites-einkommen-online.de,[],NaN,2020-12-13 15:32:15,0
4858837,zwergerlmanufaktur.de,[],NaN,2020-12-13 15:33:05,0
4859486,zwillingslinde-online.de,[],NaN,2020-12-13 15:33:55,0
4860351,zygmunt.de,[],NaN,2020-12-13 15:35:04,0


* 10.732 Zeilen haben einen NaN Wert in der Spalte A-Record

***

* es war nicht möglich ast.literal_eval auf den gesamten Datensatz anzuweden, weshalb er in Nan und not Nan Zeilen unterteilt wurde
* beide DataFrames wurden wie nötig umgewandelt und daraufhin wieder zusammengefügt

In [5]:
tmp_null=main_data[main_data["A-Record of the specific domain"].isna()]

In [6]:
tmp_null["A-Record of the specific domain"]=tmp_null["A-Record of the specific domain"].fillna({i: [] for i in tmp_null.index})

<ipython-input-6-5a23918d6e7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_null["A-Record of the specific domain"]=tmp_null["A-Record of the specific domain"].fillna({i: [] for i in tmp_null.index})


In [7]:
tmp_null["length_a"] = tmp_null["A-Record of the specific domain"].apply(lambda x: len(x))

<ipython-input-7-fa431a21043c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_null["length_a"] = tmp_null["A-Record of the specific domain"].apply(lambda x: len(x))


In [8]:
tmp_not_null=main_data[~main_data["A-Record of the specific domain"].isna()]

In [9]:
tmp_not_null["A-Record of the specific domain"]=tmp_not_null["A-Record of the specific domain"].apply(lambda x: ast.literal_eval(x))

<ipython-input-9-d5e5e526247a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_not_null["A-Record of the specific domain"]=tmp_not_null["A-Record of the specific domain"].apply(lambda x: ast.literal_eval(x))


In [10]:
tmp_not_null["length_a"] = tmp_not_null["A-Record of the specific domain"].apply(lambda x: len(x))

<ipython-input-10-6946f19ac1b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_not_null["length_a"] = tmp_not_null["A-Record of the specific domain"].apply(lambda x: len(x))


In [18]:
main_data=pd.concat([tmp_null,tmp_not_null], ignore_index=True)

In [19]:
main_data.sort_values(by=['Top level domain'],inplace=True)
main_data.reset_index(inplace=True,drop=True)

In [39]:
main_data.head(50)

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process,length_a
0,0--1.de,"[""mail.0--1.de"",""mxf993.netcup.net""]","(46.38.249.145,)",2020-12-13 15:36:05,1
1,0--2.de,"[""mxf993.netcup.net"",""mail.0--2.de""]","(212.227.212.163,)",2020-12-13 15:36:05,1
2,0-0-0-0-0.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","(80.150.6.143,)",2020-12-13 15:36:05,1
3,0-0-0-1.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","(80.150.6.143,)",2020-12-13 15:36:05,1
4,0-0.de,[],"(185.53.178.13,)",2020-12-13 15:36:05,1
5,0-01.de,"[""mail.0-01.de""]","(193.34.145.200,)",2020-12-13 15:36:05,1
6,0-1.de,"[""localhost""]","(91.195.241.137,)",2020-12-13 15:36:05,1
7,0-10.de,[],(),2020-12-13 15:36:05,0
8,0-1000.de,"[""localhost""]","(91.195.241.137,)",2020-12-13 15:36:06,1
9,0-100kmh.de,"[""smtpin.rzone.de""]","(81.169.145.95,)",2020-12-13 15:36:06,1


In [34]:
main_data["A-Record of the specific domain"].apply(tuple).value_counts()

(91.195.241.137,)                                 303204
()                                                172019
(80.150.6.143,)                                   150156
(23.236.62.147,)                                   59035
(52.58.78.16,)                                     49200
                                                   ...  
(104.27.128.34, 104.27.129.34, 172.67.216.253)         1
(217.84.82.211,)                                       1
(85.214.96.83,)                                        1
(196.196.104.195,)                                     1
(104.31.89.28, 104.31.88.28, 172.67.218.84)            1
Name: A-Record of the specific domain, Length: 450010, dtype: int64

* es ist zu erkennen, dass die rund 10k NaN Werte nun dem leeren Tuple zugeteilt werden

In [38]:
main_data["A-Record of the specific domain"]=main_data["A-Record of the specific domain"].apply(tuple)

In [42]:
main_data.drop_duplicates()

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process,length_a
0,0--1.de,"[""mail.0--1.de"",""mxf993.netcup.net""]","(46.38.249.145,)",2020-12-13 15:36:05,1
1,0--2.de,"[""mxf993.netcup.net"",""mail.0--2.de""]","(212.227.212.163,)",2020-12-13 15:36:05,1
2,0-0-0-0-0.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","(80.150.6.143,)",2020-12-13 15:36:05,1
3,0-0-0-1.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","(80.150.6.143,)",2020-12-13 15:36:05,1
4,0-0.de,[],"(185.53.178.13,)",2020-12-13 15:36:05,1
...,...,...,...,...,...
4860880,zzzw.de,"[""mail.zzzw.de""]","(104.27.159.32, 104.27.158.32, 172.67.216.93)",2020-12-13 15:36:04,3
4860881,zzzwei.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","(80.150.6.143,)",2020-12-13 15:36:04,1
4860882,zzzzzzz.de,"[""localhost""]","(91.195.241.137,)",2020-12-13 15:36:05,1
4860883,zzzzzzzzz.de,"[""localhost""]","(91.195.241.137,)",2020-12-13 15:36:05,1


* die gleiche Anzahl an Zeilen zeigt, dass es keine Duplikate in dem Datensatz gibt

### Zusätzliche Datensätze

In [ ]:
asnip4=pd.read_csv("GeoLite2-ASN-CSV_20210608/GeoLite2-ASN-Blocks-IPv4.csv")
asnip4.head()

In [ ]:
asnip4[asnip4["autonomous_system_organization"].str.match(".*?FLAR")==True]

In [ ]:
asnip6=pd.read_csv("GeoLite2-ASN-CSV_20210608/GeoLite2-ASN-Blocks-IPv6.csv")
asnip6.head()

In [ ]:
cityip4=pd.read_csv("GeoLite2-City-CSV_20210608/GeoLite2-City-Blocks-IPv4.csv")
cityip4.head()

In [ ]:
cityip6=pd.read_csv("GeoLite2-City-CSV_20210608/GeoLite2-City-Blocks-IPv6.csv")
cityip6.head()

In [ ]:
cityloc=pd.read_csv("GeoLite2-City-CSV_20210608/GeoLite2-City-Locations-de.csv")
cityloc.head()

In [ ]:
countryip4=pd.read_csv("GeoLite2-Country-CSV_20210608/GeoLite2-Country-Blocks-IPv4.csv")
countryip4.head()

In [ ]:
countryip6=pd.read_csv("GeoLite2-Country-CSV_20210608/GeoLite2-Country-Blocks-IPv6.csv")
countryip6.head()

In [ ]:
countryloc=pd.read_csv("GeoLite2-Country-CSV_20210608/GeoLite2-Country-Locations-de.csv")
countryloc.head()